# Import + Checking Environment

In [1]:
import keras as k
import tensorflow as tf
print('tensorflow: ', tf.__version__)
print('keras: ', k.__version__)

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D


Using TensorFlow backend.


tensorflow:  2.2.0
keras:  2.3.1


In [2]:
tf.test.is_built_with_cuda()

False

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
tf.test.gpu_device_name()

''

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14115195037861584279
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4892290268117421706
physical_device_desc: "device: XLA_CPU device"
]


# Preprocessing

In [6]:
# the FULL dataset

# create generator
datagen = ImageDataGenerator()

# prepare an iterators for each dataset
train_full = datagen.flow_from_directory('train_full', 
                                          target_size=(32, 32),
                                          class_mode='categorical',
                                          shuffle=True,
                                          batch_size=8)

test_full = datagen.flow_from_directory('test_full',
                                         target_size=(32, 32),
                                         class_mode='categorical',
                                         shuffle=True,
                                         batch_size=8)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [7]:
# the SMALLER dataset

# create generator
imgGen = ImageDataGenerator()

# prepare an iterators for each dataset
train_smaller = imgGen.flow_from_directory('train_smaller', 
                                            target_size=(32, 32),
                                            class_mode='categorical',
                                            shuffle=True,
                                            batch_size=8)
imgGen
test_smaller = datagen.flow_from_directory('test_smaller',
                                            target_size=(32, 32),
                                            class_mode='categorical',
                                            shuffle=True,
                                            batch_size=8)

Found 1409 images belonging to 3 classes.
Found 473 images belonging to 3 classes.


In [9]:
# the TINY dataset

# create generator
imgGen = ImageDataGenerator()

train = imgGen.flow_from_directory(directory='train_handmade',
                                               target_size=(32, 32),
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               save_to_dir='train_saved')

test = imgGen.flow_from_directory(directory='test_smaller',
                                               target_size=(32, 32),
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=True,
                                               save_to_dir='train_saved')

Found 277 images belonging to 3 classes.
Found 473 images belonging to 3 classes.


# The LeNet-5

In [10]:
# LeNet-5 - output 131
lenet5 = Sequential()

lenet5.add(Conv2D(filters=6, input_shape=(32, 32, 3), kernel_size=5, strides=1, activation='relu'))
lenet5.add(AveragePooling2D())

lenet5.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
lenet5.add(AveragePooling2D())

lenet5.add(Flatten())

lenet5.add(Dense(units=120, activation='relu'))
lenet5.add(Dense(units=84, activation='relu'))

# Output
lenet5.add(Dense(units=131, activation = 'softmax'))

lenet5.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 16)        880       
_________________________________________________________________
average_pooling2d_2 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               69240     
_________________________________________________________________
dense_2 (Dense)              (None, 84)               

In [11]:
lenet5.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
lenet5.fit(train_full, steps_per_epoch=None, epochs=100)

In [ ]:
print(lenet5.evaluate(test_full))

# Tensorboard

In [12]:
from keras.callbacks import TensorBoard
import datetime # miscellanous: to name folders

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = k.callbacks.TensorBoard(log_dir=log_dir)

In [27]:
%%time
# LeNet5 + Tensorboard + the FULL dataset
lenet5.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=[k.metrics.CategoricalAccuracy(), # Using 2 metrics
                        k.metrics.AUC()])

lenet5.fit(train_full, 
           validation_data=test_full,
           steps_per_epoch=None, epochs=15,
           callbacks=[tensorboard_callback])

Epoch 1/15
8462/8462 [==============================] - 45s 5ms/step - loss: 0.4726 - categorical_accuracy: 0.9874 - auc_1: 0.9947 - val_loss: 24.5191 - val_categorical_accuracy: 0.8932 - val_auc_1: 0.9496
Epoch 2/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5526 - categorical_accuracy: 0.9878 - auc_1: 0.9948 - val_loss: 7.9424 - val_categorical_accuracy: 0.8999 - val_auc_1: 0.9521
Epoch 3/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5212 - categorical_accuracy: 0.9898 - auc_1: 0.9955 - val_loss: 6.8745 - val_categorical_accuracy: 0.8613 - val_auc_1: 0.9331
Epoch 4/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.5245 - categorical_accuracy: 0.9902 - auc_1: 0.9956 - val_loss: 37.5403 - val_categorical_accuracy: 0.8841 - val_auc_1: 0.9434
Epoch 5/15
8462/8462 [==============================] - 46s 5ms/step - loss: 0.6070 - categorical_accuracy: 0.9904 - auc_1: 0.9956 - val_loss: 0.0000e+00 - val_categorical_accura

In [29]:
%%time
print(lenet5.evaluate(test_full))

2836/2836 [==============================] - 9s 3ms/step
[0.0, 0.8846967816352844, 0.9428174495697021]
CPU times: user 32.5 s, sys: 1.46 s, total: 33.9 s
Wall time: 9.31 s
